# Projet Lichess

_Traitements et données large échelle_

Zoé Marquis & Charlotte Kruzic

## Introduction

L'**objectif** de ce projet était d’explorer et d’analyser un ensemble de données provenant de LiChess, une plateforme d’échecs en ligne open-source qui accueille quotidiennement des millions de joueurs de tous niveaux, et qui publie les parties jouées et annotées par le moteur d’échecs Stockfish.

Nous avons utilisé ces **données** pour réaliser ce projet, plus particulièrement les parties jouées en septembre 2020, disponible sur [Kaggle](https://www.kaggle.com/datasets/noobiedatascientist/lichess-september-2020-data "Lichess September 2020 data").

Ce jeu de données contenant environ 3.74 millions de parties et 40 colonnes décrivant différents éléments des parties d'échecs (indicateurs de niveau, type de partie, ouvertures, erreurs...), nous avons utilisé Spark, un outil de traitement de données large échelle.

Nous avons commencé par répondre aux **troix questions** principales du projet en utilisant ces données, et avons donc analysé les erreurs par catégorie ELO dans les parties Blitz, calculé la probabilité de victoire en fonction de l'ouverture, et cherché à prédire le résultat d'une partie.

Enfin, nous avons élargi notre analyse en répondant à des **questions supplémentaires** notamment l'impact des ouvertures jouées sur les matchs nuls, la relation entre l'ELO et la durée d'une partie.

Les analyses réalisées dans ce projet ont permis de répondre aux questions principales tout en ouvrant des perspectives intéressantes grâce aux analyses supplémentaires. Les résultats obtenus sont accompagnés de visualisations et d’interprétations détaillées, afin d'avoir une meilleure compréhension des dynamiques des parties sur LiChess.

## Installation et importation des bibliothèques nécessaires

In [145]:
!pip install kagglehub

ERROR: Operation cancelled by user


In [ ]:
!pip install -q findspark

In [ ]:
!pip install pyspark

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import kagglehub
from collections import defaultdict

import findspark
from pyspark.sql import SparkSession

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

## Préparation des données et de l'environnement

Nous commençons par charger les données depuis Kaggle, puis nous faisons une analyse exploratoire et prétraitons les données afin d'avoir une base pour nos analyses.

### Chargement des données

In [ ]:
path = kagglehub.dataset_download("noobiedatascientist/lichess-september-2020-data")
print("Chemin vers le fichier du dataset : ", path)

In [ ]:
files = os.listdir(path)
print("Fichiers du dataset : ", files)

In [ ]:
filename = f"{path}/Sept_20_analysis.csv"
print("Nom du fichier : ", filename)

In [ ]:
# voir le contenu du .txt
filename_txt = f"{path}/Column information.txt"
with open(filename_txt, 'r') as f:
    print(f.read())

### Lancement de Spark

In [ ]:
# Imports des éléments Spark
from pyspark.sql.functions import col, when, isnull, floor, count, min as spark_min, max as spark_max
from pyspark.sql.functions import countDistinct, row_number, split, concat_ws, sum as spark_sum, rank
from pyspark.ml.functions import vector_to_array
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.stat import Correlation, ChiSquareTest
from pyspark.ml import Pipeline

In [ ]:
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
sc = spark.sparkContext
df_spark = spark.read.csv(filename, header=True, inferSchema=True)

In [ ]:
df_spark.printSchema()

In [ ]:
# nombre lignes
df_spark.count()

In [ ]:
df_spark.show(5)

### Préparation générale des données

Maintenant que les données sont chargées, nous y ajoutons les catégories ELO basées sur les plages données dans l'énoncé du projet.

En plus des 5 catégories définies dans l'énoncé, nous ajoutons "other lower bound" et "other upper bound" pour les valeurs ELO hors des plages.

#### Calcule des catégories ELO

In [ ]:
# Ajout des catégories ELO
# Catégorie ELO du joueur Noir
df_spark_plus = df_spark.withColumn("Black_ELO_category",
                              when((col("BlackElo") >= 1200) & (col("BlackElo") <= 1499), "occasional player")
                              .when((col("BlackElo") >= 1500) & (col("BlackElo") <= 1799), "good club player")
                              .when((col("BlackElo") >= 1800) & (col("BlackElo") <= 1999), "very good club player")
                              .when((col("BlackElo") >= 2000) & (col("BlackElo") <= 2399), "national and international level")
                              .when((col("BlackElo") >= 2400) & (col("BlackElo") <= 2800), "GMI, World Champions")
                              .when((col("BlackElo") < 1200), "other lower bound")
                              .otherwise("other upper bound")
                              )

# Catégorie ELO du joueur Blanc
df_spark_plus = df_spark_plus.withColumn("White_ELO_category",
                              when((col("WhiteElo") >= 1200) & (col("WhiteElo") <= 1499), "occasional player")
                              .when((col("WhiteElo") >= 1500) & (col("WhiteElo") <= 1799), "good club player")
                              .when((col("WhiteElo") >= 1800) & (col("WhiteElo") <= 1999), "very good club player")
                              .when((col("WhiteElo") >= 2000) & (col("WhiteElo") <= 2399), "national and international level")
                              .when((col("WhiteElo") >= 2400) & (col("WhiteElo") <= 2800), "GMI, World Champions")
                              .when((col("WhiteElo") < 1200), "other lower bound")
                              .otherwise("other upper bound")
                              )

# Catégorie ELO moyenne des 2 joueurs
df_spark_plus = df_spark_plus.withColumn("Avg_ELO_category", (col("BlackElo") + col("WhiteElo")) / 2)

df_spark_plus = df_spark_plus.withColumn("Avg_ELO_category",
                              when((col("Avg_ELO_category") >= 1200) & (col("Avg_ELO_category") <= 1499), "occasional player")
                              .when((col("Avg_ELO_category") >= 1500) & (col("Avg_ELO_category") <= 1799), "good club player")
                              .when((col("Avg_ELO_category") >= 1800) & (col("Avg_ELO_category") <= 1999), "very good club player")
                              .when((col("Avg_ELO_category") >= 2000) & (col("Avg_ELO_category") <= 2399), "national and international level")
                              .when((col("Avg_ELO_category") >= 2400) & (col("Avg_ELO_category") <= 2800), "GMI, World Champions")
                              .when((col("Avg_ELO_category") < 1200), "other lower bound")
                              .otherwise("other upper bound")
                              )

In [ ]:
# vérifier combien de "other ..."
other_lower_bound_black = df_spark_plus.filter(col("Black_ELO_category") == "other lower bound").count()
other_lower_bound_white = df_spark_plus.filter(col("White_ELO_category") == "other lower bound").count()
other_upper_bound_black = df_spark_plus.filter(col("Black_ELO_category") == "other upper bound").count()
other_upper_bound_white = df_spark_plus.filter(col("White_ELO_category") == "other upper bound").count()
print(f"Nombre de parties avec other lower bound pour le joueur noir : {other_lower_bound_black}")
print(f"Nombre de parties avec other lower bound pour le joueur blanc : {other_lower_bound_white}")
print(f"Nombre de parties avec other upper bound pour le joueur noir : {other_upper_bound_black}")
print(f"Nombre de parties avec other upper bound pour le joueur blanc : {other_upper_bound_white}")

In [ ]:
# répartition du nombre de parties pour avg
avg_other_lower_bound = df_spark_plus.filter(col("Avg_ELO_category") == "other lower bound").count()
avg_occasional_player = df_spark_plus.filter(col("Avg_ELO_category") == "occasional player").count()
avg_good_club_player = df_spark_plus.filter(col("Avg_ELO_category") == "good club player").count()
avg_very_good_club_player = df_spark_plus.filter(col("Avg_ELO_category") == "very good club player").count()
avg_national_international_level = df_spark_plus.filter(col("Avg_ELO_category") == "national and international level").count()
avg_GMI_World_Champions = df_spark_plus.filter(col("Avg_ELO_category") == "GMI, World Champions").count()
avg_other_upper_bound = df_spark_plus.filter(col("Avg_ELO_category") == "other upper bound").count()

# répartition du nombre de parties quand les 2 joueurs sont dans la même catégorie
same_player_other_lower_bound = df_spark_plus.filter((col("Black_ELO_category") == "other lower bound") & (col("White_ELO_category") == "other lower bound")).count()
same_player_occasional_player = df_spark_plus.filter((col("Black_ELO_category") == "occasional player") & (col("White_ELO_category") == "occasional player")).count()
same_player_good_club_player = df_spark_plus.filter((col("Black_ELO_category") == "good club player") & (col("White_ELO_category") == "good club player")).count()
same_player_very_good_club_player = df_spark_plus.filter((col("Black_ELO_category") == "very good club player") & (col("White_ELO_category") == "very good club player")).count()
same_player_national_international_level = df_spark_plus.filter((col("Black_ELO_category") == "national and international level") & (col("White_ELO_category") == "national and international level")).count()
same_player_GMI_World_Champions = df_spark_plus.filter((col("Black_ELO_category") == "GMI, World Champions") & (col("White_ELO_category") == "GMI, World Champions")).count()
same_player_other_upper_bound = df_spark_plus.filter((col("Black_ELO_category") == "other upper bound") & (col("White_ELO_category") == "other upper bound")).count()


In [ ]:
# Visualiser la répartition des catégories
categories = [
    "other lower bound", "occasional player", "good club player",
    "very good club player", "national and international level", "GMI, World Champions",
    "other upper bound"
]
avg_counts = [
    avg_other_lower_bound, avg_occasional_player, avg_good_club_player,
    avg_very_good_club_player, avg_national_international_level, avg_GMI_World_Champions,
    avg_other_upper_bound
]
same_player_counts = [
    same_player_other_lower_bound, same_player_occasional_player, same_player_good_club_player,
    same_player_very_good_club_player, same_player_national_international_level, same_player_GMI_World_Champions,
    same_player_other_upper_bound
]

df_counts = pd.DataFrame({
    'Category': categories,
    'Avg_ELO_category': avg_counts,
    'Same_Player_ELO_category': same_player_counts
})

plt.figure(figsize=(12, 6))
df_counts.set_index('Category')[['Avg_ELO_category', 'Same_Player_ELO_category']].plot(kind='bar', ax=plt.gca(), color=['skyblue', 'lightcoral'])
plt.xlabel('ELO Categories', fontsize=12)
plt.ylabel('Number of Games', fontsize=12)
plt.title('Répartition du nombre de parties pour chaque catégorie d\'ELO', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend(['Avg ELO Category', 'Same Player ELO Category'])
plt.tight_layout()
plt.show()

La répartition des données, bien que déséquilibrée entre les catégories, semble réaliste.

En effet, le nombre de parties avec des joueurs de niveaux intermédiaires est plus élevé, car ils représentent la plupart des joueurs actifs qui jouent régulièrement. Au contraire, le nombre de parties avec des joueurs ayant des niveaux extrêmes (faible ou élevé) est plus faible. Nous pouvons expliquer cela par le fait que les joueurs de bas niveau évoluent rapidement ou ne jouent pas beaucoup de parties, et les joueurs avec de hauts niveaux sont plus rares dû à la difficulté d'atteindre ces niveaux.

Ce déséquilibre naturel pourrait introduire un biais dans l'analyse, car certaines catégories sont sur/sous-représentées.

#### Récupérer le nombre de mouvements par joueur

In [ ]:
# Compter le nombre de moves pour chaque joueur (c'est White qui commence)
df_spark_plus = df_spark_plus.withColumn("white_moves",
                                         when(col("Total_moves") % 2 == 0, col("Total_moves") / 2)
                                         .otherwise(floor(col("Total_moves") / 2) + 1)
                                         )
df_spark_plus = df_spark_plus.withColumn("black_moves",
                                         when(col("Total_moves") % 2 == 0, col("Total_moves") / 2)
                                         .otherwise(floor(col("Total_moves") / 2))
                                         )

In [ ]:
df_spark_plus.select("Total_moves", "white_moves", "black_moves").show(5)

#### Exploration des données

##### **Valeurs NULL**

In [ ]:
# Calcule valeurs null par colonnes
null_counts = df_spark.select(
    *[
        count(when(col(c).isNull(), c)).alias(c)
        for c in df_spark.columns
    ]
)

null_counts.show()

##### **Colonnes Opening et ECO**

Les colonnes "Opening" et "ECO" correspondent aux ouvertures et aux codes d'ouvertures, nous regardons si elles sont en liens.

In [ ]:
# Nombre valeurs opening
print(f"Nombre de valeurs opening : {df_spark.select('Opening').distinct().count()}")
print(f"Nombre de valeurs ECO : {df_spark.select('ECO').distinct().count()}")

In [ ]:
#  Checker si une valeur de Opening = une valeur de ECO
alignment_check_1 = df_spark.groupBy("ECO").agg(countDistinct("Opening").alias("Unique_Openings"))
misaligned_rows_1 = alignment_check_1.filter(col("Unique_Openings") > 1)

In [ ]:
# Afficher les résultats
if misaligned_rows_1.count() > 0:
    print("Il existe plusieurs Openings pour un code ECO.")
    misaligned_rows_1.show(5)
    print("Nombre de lignes : ", misaligned_rows_1.count())
else:
    print("Il existe un seul Opening pour un code ECO.")

In [ ]:
# Checker si une valeur de ECO = une valeur de Opening
alignment_check_2 = df_spark.groupBy("Opening").agg(countDistinct("ECO").alias("Unique_ECOs"))
misaligned_rows_2 = alignment_check_2.filter(col("Unique_ECOs") > 1)

In [ ]:
# Afficher les résultats
if misaligned_rows_2.count() > 0:
    print("Il existe plusieurs ECO pour un code Opening.")
    misaligned_rows_2.show(5)
    print("Nombre de lignes : ", misaligned_rows_2.count())
else:
    print("Il existe un seul ECO pour un code Opening.")

Nous remarquons qu'un Opening peut avoir plusieurs ECO, et qu'un ECO peut également avoir plusieurs Opening.  
Nous allons regarder si c'est normal.

In [ ]:
# Filtrer pour les ouvertures ayant plusieurs ECO
misaligned_rows_2 = alignment_check_2.filter(col("Unique_ECOs") > 1)
opening_eco_counts = df_spark.groupBy("Opening", "ECO").agg(count("*").alias("count"))
multiple_opening_eco_counts = misaligned_rows_2.join(opening_eco_counts, on="Opening", how="inner")
multiple_opening_eco_counts.orderBy("Opening", "count", ascending=False).show(truncate=False)


Cela ne semble pas être des erreurs, il n'y a pas de ECO ou Opening largement dominant, nous allons donc garder ces éléments comme cela et les considérer comme 2 colonnes distinctes, n'ayant pas de lien particulier.

Après quelques recherche sur internet, nous avons constaté que ce sont bien deux colonnes distinctes. Pour la question 2, comme aucune indication n'est mentionnée dans l'énoncé, nous utiliserons seulement Opening.

##### **Colonnes starting_time, increment, et TimeControl**

Nous allons maintenant vérifier si les colonnes "starting_time", "increment", et "TimeControl" sont bien en accord avec la documentation.

In [ ]:
# Même nombre null pour starting_time et increment, on vérifie que c'est aligné
df_spark.filter(col("starting_time").isNull() & col("increment").isNull()).count()

In [ ]:
# Afficher les type de game quand ces 2 colonnes sont NULL
df_spark.filter(col("starting_time").isNull() & col("increment").isNull()).select("Game_type").distinct().show()

In [ ]:
# Afficher les parties avec type de jeu Correspondence et starting_time ou increment non null
df_spark.filter((col("Game_type") == "Correspondence") & (col("starting_time").isNotNull() | col("increment").isNotNull())).count()

Cela correspond à ce qui est attendu.

Maintenant, nous vérifions que TimeControl correspond bien à `starting_time+increment`.

In [ ]:
# Extraire starting_time et increment à partir de TimeControl
df_spark_check = df_spark.withColumn(
    "starting_time_extracted",
    when(col("TimeControl") != "-", split(col("TimeControl"), "\+")[0].cast("int"))
    .otherwise(None))

df_spark_check = df_spark_check.withColumn(
    "increment_extracted",
    when(col("TimeControl") != "-", split(col("TimeControl"), "\+")[1].cast("int"))
    .otherwise(None))

# Recréer TimeControl avec les colonnes extraites
df_spark_check = df_spark_check.withColumn(
    "TimeControl_reconstructed",
    when(col("starting_time_extracted").isNull() & col("increment_extracted").isNull(), "-")
    .otherwise(concat_ws("+", col("starting_time_extracted"), col("increment_extracted")))
)

# Comparer TimeControl avec la recréation
df_spark_check = df_spark_check.withColumn(
    "is_matching",
    col("TimeControl") == col("TimeControl_reconstructed")
)

# Checker les résultats
df_spark_check.select("TimeControl", "starting_time", "increment", "starting_time_extracted", "increment_extracted", "TimeControl_reconstructed", "is_matching").show(5)
mismatch_count = df_spark_check.filter(col("is_matching") == False).count()
print(f"Nombre de différences : {mismatch_count}")

Il n'y a pas de différence, cela correspond également à la documentation.

## Réponses aux questions

### Question 1

***Q1: What is the rate of blunders, errors and inaccuracies per move, per level category and on Blitz type games (Blitz type is by far the most played on these online sites). A game has two players, whose ELOs are most likely different. You will be able to classify a game into a category, either by considering the average ELO of both players, or by considering only the games where both players are in the same category.***

**Hypothèse :** Les joueurs appartenant à des catégories plus expérimentées devraient présenter un taux d'erreurs plus faible.

In [ ]:
# Filtre les parties avec le type de jeu Blitz
df_blitz = df_spark_plus.filter(col("Game_type") == "Blitz")
df_blitz.count()

#### Calcule des taux par partie

In [ ]:
# Calcule taux de blunders
df_blitz = df_blitz.withColumn("Black_blunders_rate", col("Black_blunders") / col("black_moves")) \
                   .withColumn("White_blunders_rate", col("White_blunders") / col("white_moves"))

In [ ]:
# Calcule taux d'errors
df_blitz = df_blitz.withColumn("Black_errors_rate", col("Black_mistakes") / col("black_moves")) \
                   .withColumn("White_errors_rate", col("White_mistakes") / col("white_moves"))

In [ ]:
# Calcule taux d'inaccuracies
df_blitz = df_blitz.withColumn("Black_inaccuracies_rate", col("Black_inaccuracies") / col("black_moves")) \
                   .withColumn("White_inaccuracies_rate", col("White_inaccuracies") / col("white_moves"))

#### Calcule des taux moyens par catégorie ELO (on considère le score ELO moyen des 2 joueurs)

In [ ]:
df_avg_elo_summary = df_blitz.groupBy("Avg_ELO_category").agg(
    {"Black_blunders_rate": "avg", "White_blunders_rate": "avg",
     "Black_errors_rate": "avg", "White_errors_rate": "avg",
     "Black_inaccuracies_rate": "avg", "White_inaccuracies_rate": "avg"}
).withColumnRenamed("avg(Black_blunders_rate)", "Avg_Black_blunders_rate") \
 .withColumnRenamed("avg(White_blunders_rate)", "Avg_White_blunders_rate") \
 .withColumnRenamed("avg(Black_errors_rate)", "Avg_Black_errors_rate") \
 .withColumnRenamed("avg(White_errors_rate)", "Avg_White_errors_rate") \
 .withColumnRenamed("avg(Black_inaccuracies_rate)", "Avg_Black_inaccuracies_rate") \
 .withColumnRenamed("avg(White_inaccuracies_rate)", "Avg_White_inaccuracies_rate")

In [ ]:
df_avg_elo_summary.show(5)

In [ ]:
df_avg_elo_summary_pandas = df_avg_elo_summary.toPandas()

In [ ]:
df_avg_elo_summary_pandas.isna().sum()

In [ ]:
# Ordonner les catégories de joueurs
category_order = ["other lower bound", "occasional player", "good club player", "very good club player",
                  "national and international level", "GMI, World Champions", "other upper bound"]
df_avg_elo_summary_pandas['Avg_ELO_category'] = pd.Categorical(df_avg_elo_summary_pandas['Avg_ELO_category'],  categories=category_order,  ordered=True)
df_avg_elo_summary_pandas = df_avg_elo_summary_pandas.sort_values('Avg_ELO_category')


categories = df_avg_elo_summary_pandas['Avg_ELO_category']
error_types = ['Blunders', 'Errors', 'Inaccuracies']

# Données par type d'erreur et catégorie
blunders = df_avg_elo_summary_pandas[['Avg_Black_blunders_rate', 'Avg_White_blunders_rate']].mean(axis=1)*100
mistakes = df_avg_elo_summary_pandas[['Avg_Black_errors_rate', 'Avg_White_errors_rate']].mean(axis=1)*100
inaccuracies = df_avg_elo_summary_pandas[['Avg_Black_inaccuracies_rate', 'Avg_White_inaccuracies_rate']].mean(axis=1)*100

# Matrice (erreurs x catégorie)
data = np.array([blunders, mistakes, inaccuracies]).T

In [ ]:
# Graphique
plt.figure(figsize=(14, 8))
x = np.arange(len(error_types))
bar_width = 0.1

for i, category in enumerate(categories):
    plt.bar(x + i * bar_width, data[i], width=bar_width, label=str(category))

plt.xlabel('Taux de fautes')
plt.ylabel('Pourcentage de taux moyen')
plt.title('Moyenne des taux de fautes par catégorie (Catégorie moyenne)')
plt.xticks(x + bar_width * (len(categories) - 1) / 2, error_types)
plt.legend(title="Catégorie ELO")
plt.tight_layout()
plt.show()

##### **Analyse des taux de blunders, errors et inaccuracies selon les catégories d'ELO**

Lorsque l'on analyse les taux de blunders, errors et inaccuracies en fonction des différentes catégories ELO, plusieurs tendances intéressantes émergent.  
En excluant la catégorie "Other Upper Bound", on observe une diminution progressive des taux de **bourdes** (blunders) à mesure que les catégories augmentent, avec une chute de moins en moins marquée. Cela suggère qu'il y a une amélioration du niveau des joueurs en fonction de leur catégorie ELO, bien qu'elle soit de moins en moins évidente pour les catégories les plus expérimentées.

Concernant les taux d'**erreurs** (errors), la diminution est relativement constante à travers les catégories, mais on remarque une chute plus importante lorsque l'on atteint les deux meilleures catégories ("National and International Level" et "GMI, World Champions"). Cela indique une amélioration notable dans la gestion des erreurs pour les joueurs de niveau supérieur.

Quant aux taux d'**imprécisions** (inaccuracies), la tendance reste assez stable pour les premières catégories, avec une diminution qui s'accélère à mesure que l'on approche des deux catégories les plus élevées. Cette tendance suggère également une amélioration du jeu des joueurs plus expérimentés.

En résumé, **les taux diminuent sensiblement à mesure que l'on progresse dans les catégories ELO**, avec une amélioration plus marquée pour les catégories "National and International Level" et "GMI, World Champions", ce qui reflète probablement un meilleur contrôle stratégique et une plus grande expérience des joueurs.

##### **Analyse de la catégorie "Other Upper Bound"**

Lorsque l'on considère la catégorie "Other Upper Bound", une tendance différente se dessine.  
Les taux de blunders, errors et inaccuracies semblent augmenter pour atteindre une **valeur entre celle des catégories "Good Club Player" et "Very Good Club Player"**.

Cette anomalie pourrait suggérer plusieurs pistes d'interprétation.  
Tout d'abord, il est possible que cette catégorie contienne des **données** qui ne sont **pas représentatives** du reste des catégories en raison d'un échantillon trop faible ou non nettoyé correctement.

Une autre hypothèse pourrait être que la **performance** des joueurs dans cette catégorie est **influencée** par la moyenne des ELO des deux joueurs, et non seulement par l'ELO individuel.  
Par exemple, un joueur avec un ELO élevé qui affronte un adversaire de niveau inférieur pourrait être amené à prendre plus de risques ou adopter des stratégies différentes, ce qui expliquerait certains résultats.  
De plus, il est possible qu'une partie classée dans la catégorie "Other upper bound" n'inclue qu'un seul joueur avec un ELO très élevé, ce qui fausse la moyenne des deux scores ELO et pourrait influencer l'analyse des performances.

#### Calcule des taux moyens par catégorie ELO (ici on considère seulement les parties où les joueurs sont dans la même catégorie)


In [ ]:
df_same_category = df_blitz.filter(col("Black_ELO_category") == col("White_ELO_category"))

In [ ]:
tot_blitz = df_blitz.count()
tot_same_cat = df_same_category.count()
print(f"Nombre de parties total : {tot_blitz}")
print(f"Nombre de parties avec 2 joueurs de la même catégorie : {tot_blitz}")
print(f"Pourcentage même catégorie : {tot_same_cat / tot_blitz * 100} %")

In [ ]:
df_same_category_summary = df_same_category.groupBy("Black_ELO_category").agg(
    {"Black_blunders_rate": "avg", "White_blunders_rate": "avg",
     "Black_errors_rate": "avg", "White_errors_rate": "avg",
     "Black_inaccuracies_rate": "avg", "White_inaccuracies_rate": "avg"}
).withColumnRenamed("avg(Black_blunders_rate)", "Avg_Black_blunders_rate") \
 .withColumnRenamed("avg(White_blunders_rate)", "Avg_White_blunders_rate") \
 .withColumnRenamed("avg(Black_errors_rate)", "Avg_Black_errors_rate") \
 .withColumnRenamed("avg(White_errors_rate)", "Avg_White_errors_rate") \
 .withColumnRenamed("avg(Black_inaccuracies_rate)", "Avg_Black_inaccuracies_rate") \
 .withColumnRenamed("avg(White_inaccuracies_rate)", "Avg_White_inaccuracies_rate")

In [ ]:
df_same_category_summary_pandas = df_same_category_summary.toPandas()

In [ ]:
# Ordonner les catégories de joueurs
category_order = ["other lower bound", "occasional player", "good club player", "very good club player",
                  "national and international level", "GMI, World Champions", "other upper bound"]
df_same_category_summary_pandas['Black_ELO_category'] = pd.Categorical(df_same_category_summary_pandas['Black_ELO_category'],  categories=category_order,  ordered=True)
df_same_category_summary_pandas = df_same_category_summary_pandas.sort_values('Black_ELO_category')


categories = df_same_category_summary_pandas['Black_ELO_category']
error_types = ['Blunders', 'Errors', 'Inaccuracies']

# Données par type d'erreur et catégorie (moyenne) # TODO : Voir si on fait autrement
blunders = df_same_category_summary_pandas[['Avg_Black_blunders_rate', 'Avg_White_blunders_rate']].mean(axis=1)*100
mistakes = df_same_category_summary_pandas[['Avg_Black_errors_rate', 'Avg_White_errors_rate']].mean(axis=1)*100
inaccuracies = df_same_category_summary_pandas[['Avg_Black_inaccuracies_rate', 'Avg_White_inaccuracies_rate']].mean(axis=1)*100

# Matrice (erreurs x catégorie)
data = np.array([blunders, mistakes, inaccuracies]).T

In [ ]:
# Graphique
plt.figure(figsize=(14, 8))
x = np.arange(len(error_types))
bar_width = 0.1
for i, category in enumerate(categories):
    plt.bar(x + i * bar_width, data[i], width=bar_width, label=str(category))

plt.xlabel('Type de fautes')
plt.ylabel('Pourcentage de taux moyen')
plt.title('Moyenne des taux de fautes par catégorie (Même catégorie entre les joueurs)')
plt.xticks(x + bar_width * (len(categories) - 1) / 2, error_types)
plt.legend(title="Catégorie ELO")
plt.tight_layout()
plt.show()

Pour ces parties, où les deux joueurs appartiennent à la même catégorie, **les observations sont similaires** à celles faites pour la moyenne des ELO des joueurs.

Cependant, pour la catégorie **"Other upper bound"**, on ne constate **pas de réaugmentation** des erreurs et inexactitudes, ce qui confirme que la moyenne des ELO des deux joueurs influençait les résultats dans cette catégorie.

Inversement, une légère réaugmentation des blunders est observée entre les catégories "National and International level" et "GMI, World Champions".

#### Résultats globaux

Nous observons des **résultats similaires** entre les parties où les catégories sont basées sur la moyenne des ELO des deux joueurs et celles où les deux joueurs appartiennent à la même catégorie ELO. Nous voyons, en effet, que tous **les taux de fautes ont tendance à diminuer** à mesure que la catégorie ELO augmente.

Ce résultat est en accord avec l'hypothèse initiale : Les joueurs appartenant à des catégories plus expérimentées devraient présenter un taux d'erreurs plus faible.

### Question 2

***Q2: Win probability depending on opening:***


**Hypothèse :** L'opening choisi influence significativement les chances de victoire pour les blancs et les noirs. Certains openings sont particulièrement avantageux pour les blancs tandis que d'autres peuvent mieux convenir aux noirs, en fonction du niveau des joueurs et du type de jeu (Blitz, Rapide, Classique).

Aux échecs, l'opening désigne les premiers coups joués par chaque joueur. Ces coups établissent la structure de la partie et influencent grandement les stratégies ultérieures. Les blancs ont un avantage naturel en débutant la partie, mais cet avantage peut être renforcé ou annulé en fonction de l'opening choisi par les deux joueurs.

#####  **Q2a: With which opening does White have the best chance to win, by level category (*) and by type of game (Blitz, Fast, Classic).**


**Premières observations :**   
Nous avons constaté que certaines configurations n’étaient jouées que très rarement et aboutissaient systématiquement à une victoire des Blancs.  
Cela introduit un biais et ne permet pas d’identifier correctement quel opening offre réellement le plus de chances de gagner.  
En effet, plus de 3 800 configurations présentaient une White_win_probability égale à 1.

Nous avons donc décidé de conserver uniquement les configurations avec un nombre de parties jouées supérieure à 100 afin d’obtenir des résultats plus pertinents.

Nous allons analyser les configurations possibles, c'est à dire les combinaisons de `Opening`, `White_ELO_category`, et `Game_type`.

In [ ]:
# Calculer le nombre de parties pour chaque configuration
config_game_counts = df_spark_plus.groupBy("Opening", "White_ELO_category", "Game_type").agg(count("*").alias("Total_games_count"))
config_game_counts.orderBy("Total_games_count", ascending=False).show(5)

In [ ]:
# Nombre total de configurations uniques
print(f"Nombre total de configurations possibles : {config_game_counts.count()}")
print(f"Nombre de configurations possibles pour les différents types de jeux :")
config_game_counts.groupBy("Game_type").count().orderBy("count", ascending=False).show()

In [ ]:
# Nombre d'Opening par configurations
df_spark_plus.groupBy("White_ELO_category", "Game_type").count().orderBy("count", ascending=False).show(34)

Nous pouvons voir que le nombre de parties jouées par type de jeux n'est pas répartie de façon uniforme, et que certaines configurations sont très sous représentées.

In [ ]:
# Filtrer les configurations avec plus de 100 parties jouées
filtered_configurations = config_game_counts.filter(col("Total_games_count") > 100)
filtered_configurations.orderBy("Total_games_count", ascending=False).show(5)
print(f"Nombre de configurations avec plus de 100 parties jouées : {filtered_configurations.count()}")

In [ ]:
filtered_df = df_spark_plus.join(filtered_configurations.select("Opening", "White_ELO_category", "Game_type"), on=["Opening", "White_ELO_category", "Game_type"], how="inner")

In [ ]:
# Quels sont les différentes valeurs de Game_type ?
filtered_df.select("Game_type").distinct().show()

L'énoncé précise "by type of game (Blitz, Fast, Classic)" mais on voit bien ici que c'est Blitz, "Rapid" et "Classical".

In [ ]:
# Comment sont explicités les différentes fin de partie ?
filtered_df.select("Result").distinct().show()

1-0 : Victoire des blancs  
0-1 : Victoire des noirs  
1/2-1/2 : Match nul

In [ ]:
# Récupération des parties voulues (blancs gagnes +  type de jeux Blitz, Fast, Classic)
df_white_wins = filtered_df.filter((col("Result") == "1-0") & (col("Game_type").isin(["Blitz", "Rapid", "Classical"])))
df_total_games = filtered_df.filter(col("Game_type").isin(["Blitz", "Rapid", "Classical"]))

In [ ]:
# Pour chaque ouverture, catégorie et type de jeu on calcule le nombre de victoires des blancs
df_white_wins_groupby = df_white_wins.groupBy("Opening", "White_ELO_category", "Game_type").agg(count("*").alias("White_win_count"))

# Pareil mais on calcule le total de parties jouées
df_total_games_groupby = df_total_games.groupBy("Opening", "White_ELO_category", "Game_type").agg(count("*").alias("Total_games_count"))

In [ ]:
df_white_wins_groupby.show(5)

In [ ]:
df_total_games_groupby.show(5)

In [ ]:
# Calcule de la probabilité de gagner en fonction de l'ouverture
df_opening_stats = df_white_wins_groupby.join(df_total_games_groupby, on=["Opening", "White_ELO_category", "Game_type"])
df_opening_stats = df_opening_stats.withColumn("White_win_probability", col("White_win_count") / col("Total_games_count"))

In [ ]:
df_opening_stats.show(5)

In [ ]:
df_opening_stats.count()

In [ ]:
# Y a t il toutes les combinaisons de catégorie / type de partie ?
df_opening_stats.groupBy("White_ELO_category", "Game_type").count().orderBy("count", ascending=False).show(truncate=False)

Nous pouvons voir que le nombre d'Opening différents (count) pour les combinaisons de catégories ELO et type de jeux n'est pas répartie uniformément, et certaines sont même absentes.

Cependant, dans le jeu de données initiale, ces configurations étaient très sous représentées. Notamment :
- GMI, World Champions - Rapid -  2163
- other upper bound - Blitz - 301
- GMI, World Champions - Classical - 232
- other upper bound - Rapid - 8
- other upper bound - Classical - 1

In [ ]:
# Récupération du meilleur opening pour chaque catégorie de joueur et type de partie
window_spec = Window.partitionBy("White_ELO_category", "Game_type").orderBy(col("White_win_probability").desc())
best_openings = df_opening_stats.withColumn("rank", rank().over(window_spec))
best_openings = best_openings.filter(col("rank") == 1).select("White_ELO_category", "Game_type", "Opening", "White_win_probability")

###### Résultats

Nous affichons maintenant pour chaque catégorie ELO et type de jeu, l'opening permettant le plus de gagner pour les blancs.

In [ ]:
best_openings.orderBy("White_ELO_category", "Game_type").show(17)

In [ ]:
best_openings_pandas = best_openings.toPandas()

In [ ]:
best_openings_pandas["GameType_Category"] = (best_openings_pandas["Game_type"] + " | " + best_openings_pandas["White_ELO_category"])

# Table pivot pour voir le meilleur opening pour chaque configuration
pivot_table = best_openings_pandas.pivot_table(
    index="GameType_Category",
    columns="Opening",
    values="White_win_probability"
)

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Meilleure opening (plus grande probabilité de victoire) des Blancs pour une catégorie et un Game Type")
plt.xlabel("Opening")
plt.ylabel("Type de jeu et Catégorie ELO")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

Nous pouvons voir que l'opening obtenant un meilleur résultat est généralement différent entre les configurations. Cela pourrait montrer qu'il y a un lien entre la victoire, le type de jeu, le niveau du joueur et l'opening choisi.

Plus la probabilité de victoire associée à un opening est proche de 0,5, plus la chance de gagner avec cet opening est réduite, même si elle reste légèrement favorable (> 0,5). Cela reflète une situation où l'opening ne confère qu'un léger avantage, sans être déterminant. À l'inverse, une probabilité de victoire élevée, comme 0,83 dans le cas du "King Knight Opening" pour les joueurs de très bon niveau en Blitz, indique un opening particulièrement efficace pour maximiser les chances de victoire des blancs dans ce contexte spécifique.

##### **Q2b: same question with black. You don't need to write again the same but only the results with black.**

In [ ]:
# Calculer le nombre de parties pour chaque configuration
config_game_counts = df_spark_plus.groupBy("Opening", "Black_ELO_category", "Game_type").agg(count("*").alias("Total_games_count"))

In [ ]:
# Filtrer les configurations avec plus de 100 parties jouées
filtered_configurations = config_game_counts.filter(col("Total_games_count") > 100)

In [ ]:
filtered_df = df_spark_plus.join(filtered_configurations.select("Opening", "Black_ELO_category", "Game_type"), on=["Opening", "Black_ELO_category", "Game_type"], how="inner")

In [ ]:
df_black_wins = filtered_df.filter((col("Result") == "0-1") & (col("Game_type").isin(["Blitz", "Rapid", "Classical"])))
df_total_games = filtered_df.filter(col("Game_type").isin(["Blitz", "Rapid", "Classical"]))

In [ ]:
df_black_wins_groupby = df_black_wins.groupBy("Opening", "Black_ELO_category", "Game_type").agg(count("*").alias("Black_win_count"))
df_total_games_groupby = df_total_games.groupBy("Opening", "Black_ELO_category", "Game_type").agg(count("*").alias("Total_games_count"))

In [ ]:
df_opening_stats = df_black_wins_groupby.join(df_total_games_groupby, on=["Opening", "Black_ELO_category", "Game_type"])
df_opening_stats = df_opening_stats.withColumn("Black_win_probability", col("Black_win_count") / col("Total_games_count"))

Nous affichons maintenant pour chaque catégorie ELO et type de jeu, l'opening (des blancs) permettant le plus de gagner pour les noirs.

In [ ]:
window_spec = Window.partitionBy("Black_ELO_category", "Game_type").orderBy(col("Black_win_probability").desc())
best_openings = df_opening_stats.withColumn("rank", rank().over(window_spec))
best_openings = best_openings.filter(col("rank") == 1).select("Black_ELO_category", "Game_type", "Opening", "Black_win_probability") # Rank pour garder les égalités
best_openings.orderBy("Black_ELO_category", "Game_type").show(truncate=False)

In [ ]:
best_openings_pandas = best_openings.toPandas()

In [ ]:
best_openings_pandas["GameType_Category"] = (best_openings_pandas["Game_type"] + " | " + best_openings_pandas["Black_ELO_category"])
pivot_table = best_openings_pandas.pivot_table(
    index="GameType_Category",
    columns="Opening",
    values="Black_win_probability"
)

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Meilleure opening (plus grande probabilité de victoire) des Noirs pour une catégorie et un Game Type")
plt.xlabel("Opening")
plt.ylabel("Type de jeu et Catégorie ELO")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

Nous observons que les openings offrant le plus de chances de victoire aux noirs sont différents de ceux favorisant les blancs. De plus, l'opening associé aux meilleures chances de victoire varie également en fonction des configurations, qu'il s'agisse du type de jeu (Blitz, Rapide, Classique) ou du niveau des joueurs.

Par exemple, dans les parties classiques pour les joueurs d'un niveau "other lower bound", l'opening "Queen's Pawn Game; Chigorin Variation" affiche une probabilité de victoire de 0,52, ce qui suggère un léger avantage pour les noirs, bien que réduit.

Nous remarquons également que certains openings semblent efficaces à travers différents types de jeu et niveaux, démontrant une certaine polyvalence. Cela souligne l'importance de l'adaptation stratégique des joueurs selon le contexte pour maximiser leurs chances de succès.

#### Réponse à l'hypothèse :
- Les données montrent que l'opening choisi influence effectivement les chances de victoire, et cet impact varie selon plusieurs facteurs :

##### Différence entre blancs et noirs :
- Certains openings favorisent nettement les blancs, tandis que d'autres conviennent davantage aux noirs. Cela illustre une dynamique stratégique où les choix initiaux de chaque camp influencent fortement l'évolution de la partie.

##### Variabilité selon les configurations :
- Dans les parties Blitz, par exemple, des openings spécifiques comme le "King's Knight Opening" pour les blancs offrent des probabilités de victoire élevées, atteignant 0,83. Cela suggère que la rapidité du jeu peut accentuer l'efficacité de certains openings.
- Dans des catégories comme "other lower bound" en classique, des openings comme "Queen's Pawn Game; Chigorin Variation" n'apportent qu'un avantage limité (0,52), montrant une moindre influence stratégique à ce niveau.

##### Proximité avec 0,5 :
- Plus la probabilité de victoire liée à un opening est proche de 0,5, moins cet opening semble décisif. Cela signifie qu'il joue un rôle plus neutre dans l'issue de la partie, même s'il reste un léger avantage pour l'un des camps.

##### Polyvalence de certains openings :
- Certains openings apparaissent comme efficaces à travers différents types de jeux et niveaux de joueurs, indiquant qu'ils peuvent être des choix stratégiques universels.

### Question 3

***Q3: (difficult). Does a line of data in the file predict the outcome of the game (column Result), and with what
probability? In other words, can any of the variables, such as the number of errors (mistakes, blunders, inacurracies,
ts_blunders), the difference in ELO between the two players, etc., explain the outcome (win/loss)? You are free to
define explain as you wish. It can be a correlation, linear or not, or any other relationship that allows this prediction.  
Note that the ELO is itself computed from a probability (normal distribution) of victory depending on the difference
in ELO of the two players. For instance, for a difference of 100 ELO points, the higher ranked player is expected to
win with probability 0.64. For a 200 points difference, it is 0.76.  
As we have more data than the ELO difference, your prediction should be more accurate than that.***

Pour répondre à cette question, nous avons adopté une approche combinant des analyses exploratoires et des techniques de machine learning afin d'évaluer la capacité des variables à expliquer ou prédire le résultat d'une partie (colonne *Result*). Cependant, la grande quantité de données disponibles a posé des défis significatifs, notamment en termes de temps d'entraînement pour certains modèles sophistiqués, comme les forêts aléatoires (*Random Forests*) ou les arbres boostés (*Gradient Boosted Trees*), même en réduisant l'échantillon à un pourcentage aléatoire des données.

Face à ces contraintes, nous avons opté pour un modèle de régression multinomiale, qui s'est avéré bien plus rapide à entraîner tout en offrant des performances acceptables sur un sous-échantillon de 1 % des données. Cela a toutefois nécessité une adaptation spécifique de la préparation des données. En parallèle, pour mieux comprendre les relations entre les variables et le résultat, nous avons complété l'analyse par des mesures de corrélation, de covariance et des tableaux de contingence (avec test de chi2), afin de capturer des liens potentiellement explicatifs ou prédictifs entre les caractéristiques comme les erreurs (*mistakes*, *blunders*, etc.), la différence d'ELO entre les joueurs, et d'autres variables pertinentes.


#### Hypothèse :
L'hypothèse principale est que la différence d'ELO est un facteur significatif pour prédire l'issue de la partie, conformément à la théorie sous-jacente à son calcul (probabilité basée sur une distribution normale).

Cependant, étant donné la richesse des données, d'autres facteurs pourraient également jouer un rôle dans la prédiction de l'issue de la partie. Ces facteurs incluent :

- Nombre d'erreurs (blunders, mistakes, inaccuracies, ts_blunders) : des erreurs fréquentes devraient augmenter les chances de défaite.
- Nombre de coups totaux (Total_moves) : des parties plus longues peuvent refléter un jeu plus égalisé ou stratégique.
- Niveau de jeu (type de partie : Blitz, Classique, etc.) : les parties rapides pourraient amplifier l'effet des erreurs.
- Autres statistiques liées au jeu : comme les renversements de partie (Game_flips), etc.

#### Préparation des données

In [ ]:
df_spark_plus.show(5)

In [ ]:
# Ajouter la colonne de différence d'ELO
df_spark_plus = df_spark_plus.withColumn("ELO_diff", col("WhiteELO") - col("BlackELO"))

##### **Suppression des colonnes non nécessaires**

Nous allons supprimer certaines colonnes du jeu de données qui ne sont pas pertinentes pour la prédiction de la colonne `Result` et/ou qui pourrait introduire un biais.

<ins>Suppression des colonnes étant des conséquences du résultat:</ins>
* `BlackRatingDiff` :  Variation du classement ELO du joueur noir après la partie
* `WhiteRatingDiff` : Variation du classement ELO du joueur blanc après la partie

Ces colonnes reflètent directement l'issue de la partie et ne peuvent donc pas être utilisées comme des variables explicatives pour prédire le résultat. Cela nous évite aussi de devoir gérer les valeurs NULL présentes dans ces colonnes.

<ins>Suppression des colonnes n'apportant pas d'informations pertinentes :</ins>
* `GAME` : Identifiant unique de la partie
* `Date`: Date à laquelle la partie a été jouée
* `Site`: URL de la partie  
* `TimeControl` : Temps de jeu en secondes (temps initial + incrément)
* `UTCTime` : Heure à laquelle la partie a été jouée
* `Event` : Evenement où la partie a été jouée

En plus d'être non pertinentes, certaines de ces données ne sont pas standardisées, cela nous évite donc des traitements supplémentaires non nécessaires.

Supprimer les colonnes que nous avons calculées (informations redondantes) :
- `Black_ELO_category`
- `White_ELO_category`
- `Avg_ELO_category`

In [ ]:
# Combien d'instances pour Game_type
df_spark_plus.groupBy("Game_type").count().show()

In [ ]:
# Supprimer Game_type = Correspondance pour pouvoir garder starting_time et increment
# (où les données peuvent être manquantes)
df_spark_plus = df_spark_plus.filter(col("Game_type") != "Correspondence")

In [ ]:
# Suppression des colonnes
df_preparation = df_spark_plus.drop("BlackRatingDiff", "WhiteRatingDiff", "GAME", "Date", "Site", "TimeControl", "UTCTime", "Event",  "Black_ELO_category", "White_ELO_category", "Avg_ELO_category")

In [ ]:
df_preparation.show(5)

In [ ]:
# Combien de valeurs nulles par colonne ?
df_preparation.select([count(when(col(c).isNull(), c)).alias(c) for c in df_preparation.columns]).show()


In [ ]:
# Pas de valeur null, c'est parfait

Nous réalisons l'analyse de corrélation, de covariance et des tableaux de contingence avant de finaliser la préparation des données, car la normalisation, la standardisation et l'encodage des données catégorielles peuvent influencer l'interprétation des relations entre les variables. En effet, ces techniques de prétraitement modifient l'échelle ou la représentation des données, ce qui peut fausser les résultats des analyses de corrélation ou de covariance si elles sont effectuées après ces transformations.


Pourquoi cela est important :

Corrélation et Covariance :
- La corrélation mesure la force et la direction d'une relation linéaire entre deux variables. Elle peut être affectée par la mise à l'échelle des données. Si ces transformations sont faites après l'analyse de corrélation, il devient plus difficile d'interpréter les relations d'origine.
- La covariance, bien que similaire à la corrélation, n'est pas dimensionnée. Elle peut être influencée par les unités de mesure des variables, ce qui peut fausser l'interprétation si les données ne sont pas préparées correctement.
- **NB** : Nous avons tout de même besoin d'encoder Result en Result_index avant l'analyse de corrélation et de covariance.

Encodage des données catégorielles :
- Lorsque nous avons des colonnes catégorielles, il est nécessaire de les encoder. L'encodage peut introduire des relations implicites ou artificielles entre les variables. Cela peut affecter la façon dont les relations entre les catégories sont perçues dans les analyses. Il est donc important d'encodé correctement ces variables avant d'exécuter l'analyse, pour éviter d'introduire de fausses relations ou de perdre des informations importantes.

In [ ]:
# Nombre de valeurs par Result
df_preparation.groupBy("Result").count().show()

Il y a seulement 72 valeurs indéfinies dans la colonnes Result, nous allons supprimer ces parties d'échec (ces lignes), car elles n'ont pas d'intérêts et sont en trop faible nombre pour apporter un réel résultat à notre analyse.

In [ ]:
df_preparation = df_preparation.filter(col("Result") != "*")

#### Table de contingence

In [ ]:
# Sélectionner toutes les colonnes catégorielles dans le DataFrame
cat_columns = [col for col, dtype in df_preparation.dtypes if dtype == 'string']
cat_columns

In [ ]:
# Fonction pour créer la table de contingence
def create_contingency_table(data, col1, col2):
    return data.groupBy(col1, col2).agg(F.count('*').alias('count'))

# Exclure 'Result' de cat_columns
cat_columns = [col for col in cat_columns if col != 'Result']

# Liste pour stocker les résultats
contingency_tables = []

# Boucle pour générer la table de contingence pour chaque paire de colonnes catégorielles
for col in cat_columns:
    # Créer la table de contingence
    contingency_table = create_contingency_table(df_preparation, col, 'Result')
    contingency_table_ord = contingency_table.orderBy(col, 'Result')
    print(f"Contingency Table for {col} and Result :")
    contingency_table_ord.show()

    # Convertir la table de contingence en DataFrame Pandas pour l'analyse
    contingency_df = contingency_table.toPandas()

    # Calculer les proportions de chaque combinaison par rapport au total
    contingency_df['Proportion'] = contingency_df['count'] / contingency_df['count'].sum()

    # Ajouter la table de contingence à la liste
    contingency_tables.append({
        "Variable": col,
        "Contingency Table": contingency_df
    })

    # S'assurer que chaque combinaison existe, remplir les valeurs manquantes avec 0
    contingency_df = contingency_df.pivot_table(index=col, columns='Result', values='Proportion', aggfunc='sum', fill_value=0)

    # Visualisation de la table de contingence
    plt.figure(figsize=(10, 6))
    contingency_df.plot(kind='bar', stacked=True)
    plt.title(f"Proportions of {col} and Result")
    plt.xlabel(col)
    plt.ylabel("Proportion")
    plt.legend(title="Result")
    plt.show()

# Résumé des tables de contingence
for result in contingency_tables:
    print(f"Summary for {result['Variable']}:")
    print(result['Contingency Table'].head())  # Affiche les premières lignes pour une vue d'ensemble
    print("\n")


Les analyses des tables de contingence pour les relations entre ECO et Result, Opening et Result, Termination et Result, ainsi que Game_type et Result ne révèlent pas de différences significatives en termes de proportions, avec une distribution globalement équilibrée entre les victoires des Blancs, des Noirs et un nombre notablement plus faible de matchs nuls.

In [ ]:
# Sélectionner toutes les colonnes catégorielles dans le DataFrame qui ne sont pas Result
cat_columns = [col for col, dtype in df_preparation.dtypes if dtype == 'string' and col != 'Result']

# Indexation des colonnes catégorielles
indexers = [
    StringIndexer(inputCol=col_name, outputCol=col_name + "_index")
    for col_name in cat_columns + ['Result']
]

# Créer un VectorAssembler pour assembler les colonnes en un vecteur
assembler = VectorAssembler(
    inputCols=[col_name + "_index" for col_name in cat_columns],
    outputCol="features"
)

# Créer le pipeline
pipeline = Pipeline(stages=indexers + [assembler])

# Appliquer le pipeline pour transformer les données
df_transformed = pipeline.fit(df_preparation).transform(df_preparation)

# Liste pour stocker les résultats
chi_results = []

# Boucle pour générer la table de contingence et effectuer le test chi-deux pour chaque paire de colonnes catégorielles
for col in cat_columns:
    # Créer un DataFrame avec les colonnes à tester (en utilisant "features" comme colonne d'entrée)
    df_chi = df_transformed.select("features", col + "_index")

    # Effectuer le test de chi-deux
    chi_result = ChiSquareTest.test(df_chi, "features", col + "_index").head()

    # Afficher le résultat du test de chi-deux
    print(f"Chi-Square Test for {col} and Result:")
    print(f"Chi-Square Statistic: {chi_result[0]}, p-value: {chi_result[1]}")

    # Ajouter les résultats dans la liste
    chi_results.append({
        "Variable": col,
        "Chi-Square Statistic": chi_result[0],
        "p-value": chi_result[1]
    })

# Créer un DataFrame pour les résultats
chi_results_df = spark.createDataFrame(chi_results)

# Afficher les résultats
chi_results_df.show()



Les résultats des tests du chi-deux obtenus montrent les valeurs statistiques et les p-values pour différentes variables catégorielles par rapport à la variable cible Result.

1. **ECO et Result**

Chi-Square Statistic : [0.0, 0.0, 0.0, 0.0]  
p-value : [241081, 1368908, 1473, 1473]  
L'absence de valeurs supérieures à zéro dans les statistiques du chi-deux (valeurs égales à 0) signifie qu'il n'y a pas d'association entre les catégories de la variable ECO et la variable cible Result. Cela suggère que les différentes catégories de ECO sont indépendantes de Result, du moins selon ce test.

Les p-values élevées (241081, 1368908, etc.) confirment l'absence de relation statistique significative entre les deux variables, car une p-value élevée (généralement > 0.05) indique une faible probabilité que l'association observée soit due au hasard.

2. **Opening et Result**

Chi-Square Statistic : [0.0, 0.0, 0.0, 0.0]  
p-value : [1368908, 7772944, 8364, 8364]  
Comme pour ECO, les statistiques du chi-deux de Opening sont égales à 0, ce qui indique qu'il n'y a aucune association significative entre cette variable et Result. Les p-values très élevées renforcent cette conclusion : la probabilité que l'absence d'association soit due au hasard est très faible.

3. **Termination et Result**

Chi-Square Statistic : [0.0, 0.0, 0.0, 0.0]  
p-value : [1473, 8364, 9, 9]  
Ici encore, la statistique du chi-deux est nulle, ce qui suggère qu'il n'y a pas d'association significative entre Termination et Result. Cependant, les p-values sont légèrement plus petites (9 et 8364), ce qui peut indiquer des zones où une association pourrait potentiellement être présente, mais elles restent suffisamment élevées pour indiquer qu'il n'y a pas de relation forte.

4. **Game_type et Result**

Chi-Square Statistic : [0.0, 0.0, 0.0, 0.0]  
p-value : [1473, 8364, 9, 9]  
Comme les résultats pour Termination, la statistique du chi-deux est de 0, ce qui ne montre aucune association significative entre Game_type et Result. Les p-values sont similaires à celles de Termination, ce qui confirme qu'il n'y a pas de lien important entre ces variables.

**Conclusion générale :**

Les résultats montrent que pour ces variables (ECO, Opening, Termination, Game_type), les tests de chi-deux ne révèlent aucune relation statistiquement significative avec la variable Result. En effet, les statistiques du chi-deux sont toutes nulles (0.0), ce qui suggère que la distribution des catégories de ces variables est indépendante de la variable cible Result. Les p-values élevées corroborent ce constat, car une p-value élevée indique que l'on ne peut pas rejeter l'hypothèse nulle (indépendance entre les variables).

Cela signifie que aucune des variables testées (ECO, Opening, Termination, Game_type) n'a de lien évident avec Result dans notre jeu de données, du moins selon le test statistique du chi-deux effectué.

On encode Result en donnée numérique.

In [ ]:
# Result : colonne cible, 3 valeurs possibles
from pyspark.sql.functions import when, col # très capricieux, laissé l'import

df_preparation = df_preparation.withColumn(
    "Result_index",
    when(col("Result") == "1-0", 0)
    .when(col("Result") == "0-1", 2)
    .when(col("Result") == "1/2-1/2", 1) # nul "moyenne" des deux autres scénarios
)

# Vérification
df_preparation.select("Result", "Result_index").distinct().show()

#### Corrélation

La corrélation mesure la force et la direction de la relation linéaire entre deux variables.

- Une corrélation proche de +1 ou -1 indique une forte relation linéaire.
- Une corrélation proche de 0 indique peu ou pas de relation linéaire.


In [ ]:
# Quelles sont les colonnes numériques ?
numeric_cols = [col[0] for col in df_preparation.dtypes if col[1] in ["int", "double"]]
print(f'Les colonnes numériques sont : {numeric_cols}')

In [ ]:
# Assembler les colonnes numériques en un seul vecteur
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="numeric_features")
assembled_data = assembler.transform(df_preparation)

# Calculer la matrice de corrélation
correlation_matrix = Correlation.corr(assembled_data, "numeric_features").head()[0]

# Convertir en DataFrame pour un affichage clair
correlation_array = np.array(correlation_matrix.toArray())
correlation_df = pd.DataFrame(correlation_array, columns=numeric_cols, index=numeric_cols)
print(correlation_df)

In [ ]:
# Créer une heatmap avec seaborn pour une meilleure visualisation
plt.figure(figsize=(10, 8))

# Utiliser une palette de couleurs allant de -1 à 1 (avec 0 en blanc)
sns.heatmap(correlation_df, annot=True, cmap='RdBu', center=0, vmin=-1, vmax=1, fmt=".2f", linewidths=0.5)

# Ajouter un titre
plt.title("Matrice de Corrélation", fontsize=16)

# Afficher la heatmap
plt.show()

In [ ]:
# Sélectionner uniquement la dernière ligne (ici 'Result_index')
result_corr = correlation_df.loc['Result_index']

# Diviser en groupes de 5 colonnes
columns = correlation_df.columns
step = 5

for i in range(0, len(columns), step):
    # Sélectionner un sous-ensemble de colonnes
    subset_columns = columns[i:i + step]
    result_corr_subset = result_corr[subset_columns].to_frame().T

    # Afficher les colonnes sélectionnées
    print(result_corr_subset)

    # Créer une heatmap pour le sous-ensemble
    plt.figure(figsize=(10, 1))  # Ajuster la taille pour le sous-ensemble
    sns.heatmap(result_corr_subset, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1, fmt=".2f", linewidths=0.5)
    plt.title(f"Corrélation avec Result_index (Colonnes {i+1} à {i+len(subset_columns)})", fontsize=14)
    plt.show()


##### **Variables liées à l'ELO**
- `BlackElo` (0.029) : Une faible corrélation positive indique que lorsque l'ELO du joueur noir augmente, il y a une légère tendance à une victoire pour Black (classe 2), mais cet effet est négligeable. (Peut-être pas une vistoire de noir, mais on tend alors vers 1 ou 2, match nul ou victoire de noir.)
- `WhiteElo` (-0.041) : Une faible corrélation négative suggère qu'un ELO élevé pour White favorise légèrement la victoire de White (classe 0), mais l'effet reste marginal.
- `ELO_diff` (-0.167) : Une corrélation négative modérée montre que lorsque la différence d'ELO augmente (en faveur de White), la probabilité de victoire pour White (classe 0) augmente, ce qui est intuitif.

##### **Variables temporelles**
- `starting_time` (-0.001) et `increment` (-0.000) : Les corrélations quasi nulles montrent que ni le temps de départ ni l'incrément n'ont d'influence significative sur le résultat de la partie.

##### **Variables sur le nombre de coups**
- `Total_moves` (0.027) : Une faible corrélation positive indique que les parties avec plus de coups sont légèrement associées à des résultats favorisant les matchs nuls (classe 1) ou les victoires de noir (classe 2), mais l'effet est très faible.

##### **Blunders**
- `Black_blunders` (-0.217) : Une corrélation négative modérée montre que plus le joueur noir commet des blunders, moins il est probable qu'il gagne (classe 2), ce qui est attendu.
- `White_blunders` (0.238) : Une corrélation positive modérée montre que plus le joueur blanc fait de blunders, plus il est probable que White perde, ce qui favorise les victoires de Black (classe 2).

##### **Mistakes et inaccuracies**
- `Black_mistakes` (-0.105) et `Black_inaccuracies` (-0.111) : Des corrélations négatives faibles montrent que les erreurs mineures des noirs réduisent leurs chances de victoire, mais pas de manière aussi significative que les blunders.
- `White_mistakes` (0.136) et `White_inaccuracies` (0.143) : Des corrélations positives faibles montrent que les erreurs des blancs augmentent la probabilité de victoire pour Black.

##### **Inferior moves (mauvais coups globaux)**
- `Black_inferior_moves` (-0.206) : Une corrélation négative modérée montre que des mauvais coups fréquents chez les noirs réduisent leurs chances de victoire (classe 2).
- `White_inferior_moves` (0.245) : Une corrélation positive modérée montre que des mauvais coups fréquents chez les blancs augmentent les chances pour Black de gagner (classe 2).

##### **Time-sensitive moves (erreurs sous pression de temps)**
- `Black_ts_blunders` (-0.120) et `Black_ts_mistakes` (-0.089) : Ces corrélations montrent que les erreurs des noirs sous pression temporelle réduisent leurs chances de victoire, mais l'effet est modéré.
- `White_ts_blunders` (0.136) et `White_ts_mistakes` (0.104) : Ces corrélations positives montrent que les erreurs des blancs sous pression temporelle augmentent les chances pour Black de gagner (classe 2).

##### **Long moves et bad long moves**
- `Black_long_moves` (-0.121) et `Black_bad_long_moves` (-0.126) : Ces corrélations négatives faibles indiquent que des mauvais coups longs chez les noirs réduisent légèrement leurs chances de victoire.
- `White_long_moves` (0.127) et `White_bad_long_moves` (0.132) : Ces corrélations positives faibles montrent que les mauvais coups longs chez les blancs augmentent légèrement les chances pour Black de gagner.

##### **Game flips (changement de dynamique)**
- `Game_flips` (0.022) et `Game_flips_ts` (0.010) : Ces corrélations très faibles montrent que les retournements de situation dans la partie n'ont presque aucun impact sur le résultat final.

##### **Synthèse des principaux facteurs :**
- Différence d'ELO (`ELO_diff`) a une influence notable, avec une tendance claire : un ELO plus élevé favorise la victoire du joueur plus fort.
- Blunders (`Black_blunders` et `White_blunders`) ont l'effet le plus significatif sur le résultat. Les blunders des noirs diminuent leurs chances, tandis que ceux des blancs augmentent les chances de victoire pour Black.
- Inferior moves (Black et White) suivent une tendance similaire aux blunders, bien que leur impact soit légèrement plus faible.
- Mistakes et inaccuracies ont un impact moindre, mais leur tendance est cohérente avec celle des blunders.
- Facteurs temporels et coups longs ont peu d'effet significatif sur le résultat.


##### **Conclusion**
Les résultats confirment que la performance est fortement liée aux erreurs majeures et à l'ELO des joueurs, tandis que les autres variables comme le temps ou les retournements de jeu ont une influence marginale.


#### Covariance

- Une covariance positive indique que les deux variables augmentent ensemble.
- Une covariance négative indique qu'une variable augmente tandis que l'autre diminue.

In [ ]:
# Liste des colonnes numériques sauf "Result"
numeric_cols_without_result = [col for col in numeric_cols if col != 'Result_index']

data = df_preparation

# Calculer la covariance entre 'Result' et chaque autre colonne numérique
cov_result = []
for col in numeric_cols_without_result:
    print(f"Calcul de {col}")
    covariance = data.stat.cov('Result_index', col)
    cov_result.append(covariance)

# Créer un DataFrame pour afficher les résultats
cov_result_df = pd.DataFrame(cov_result, columns=['Covariance'], index=numeric_cols_without_result)

# Afficher les covariances
print(cov_result_df)


In [ ]:
# Créer une heatmap pour la matrice de covariance
plt.figure(figsize=(10, 8))  # Ajuster la taille pour mieux visualiser

# Utiliser une palette de couleurs pour les covariances
sns.heatmap(cov_result_df, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, center=0, vmin=-cov_result_df.max().max(), vmax=cov_result_df.max().max())

# Ajouter un titre
plt.title("Matrice de Covariance", fontsize=16)

# Afficher la heatmap
plt.show()

##### **Covariances importantes avec Result_index :**
- `BlackElo` (9.88) : Une covariance positive suggère que l'ELO plus élevé du joueur noir est associé à une plus grande probabilité de victoire pour Black (classe 2). Cela est logique, car un joueur avec un ELO plus élevé est plus susceptible de gagner, mais l'effet n'est pas très fort.
- `WhiteElo` (-14.05) : La covariance négative indique que l'ELO plus élevé du joueur blanc est associé à une probabilité plus faible de match nul ou de victoire pour Black. Cela montre une relation inverse : un joueur blanc plus fort (plus élevé en ELO) augmente la probabilité de victoire pour White (classe 0).
- `Total_moves` (0.72) : La covariance positive montre que plus le nombre de coups est élevé, plus la probabilité de match nul (classe 1) augmente. Cela pourrait indiquer que les matchs avec plus de coups sont souvent plus équilibrés et ont une probabilité plus élevée de se terminer par un match nul. (Ceci dit, la probabilité de victoire de Black augmente par la même occasion.)
- `Black_blunders` (-0.42) et `White_blunders` (0.46) : Les blunders des noirs et des blancs ont une covariance relativement forte avec le résultat, suggérant que des erreurs (blunders) des noirs diminuent la probabilité de victoire pour Black (classe 2), tandis que les blunders des blancs augmentent la probabilité de victoire pour White (classe 0). Les blunders sont donc un facteur significatif dans le résultat de la partie.
- `Black_inferior_moves` (-0.88) et `White_inferior_moves` (1.06) : Les mauvais coups (moves inférieurs) des noirs sont associés négativement avec le résultat pour Black, tandis que les mauvais coups des blancs augmentent la probabilité de victoire de White. Cela montre que les erreurs stratégiques jouent un rôle important dans l'issue du match.
- `ELO_diff` (-23.93) : Une forte covariance négative indique que la différence d'ELO a un impact important sur le résultat. Lorsque la différence d'ELO entre les joueurs est importante, cela favorise la victoire de White (classe 0) ou de Black (classe 2), selon qui a le plus grand ELO. Cela est cohérent avec l'idée que des différences d'ELO plus élevées augmentent la probabilité que le joueur avec l'ELO le plus élevé gagne.

##### **Covariances faibles ou peu significatives :**
- `starting_time` (-0.42) et `increment` (-0.0006) : Ces valeurs faibles indiquent que le temps de départ et l'incrément n'ont qu'un faible impact sur le résultat du match. Cela soutient l'idée que les caractéristiques temporelles, comme le temps de départ ou les incréments, influencent peu le résultat final.
- `Black_mistakes` (-0.24) et `White_mistakes` (0.31) : Les erreurs sont légèrement corrélées avec le résultat, mais l'impact est modéré. Les erreurs des noirs réduisent légèrement la probabilité de victoire de Black, tandis que les erreurs des blancs ont un effet inverse.
- `Black_ts_moves` (-0.42) et `White_ts_moves` (0.53) : Les mouvements temporels (moves dans la time control) ont une corrélation faible, mais légèrement plus marquée du côté des blancs, où des mouvements dans le cadre du contrôle du temps pourraient favoriser leur victoire.
- `Black_long_moves` (-0.13) et `White_long_moves` (0.14) : Les mouvements longs n'ont qu'une faible covariance avec le résultat, bien que les mouvements longs des blancs semblent légèrement augmenter la probabilité de leur victoire.

##### **Autres variables et facteurs supplémentaires :**
- `Black_bad_long_moves` (-0.08) et `White_bad_long_moves` (0.08) : Bien que faibles, ces covariances suggèrent que les mouvements longs "mauvais" des noirs sont légèrement corrélés avec une diminution de leur probabilité de victoire, tandis que ceux des blancs semblent avoir l'effet inverse, mais de manière marginale.
- `Game_flips` (0.10) et `Game_flips_ts` (0.01) : Ces valeurs indiquent que les flips du jeu n'ont qu'un très faible impact sur le résultat du match.

##### **Conclusion générale :**
- L'ELO et la différence d'ELO ont un impact fort sur le résultat du match. Un ELO plus élevé pour l'un des joueurs diminue la probabilité de match nul et favorise la victoire de ce joueur.
- Les blunders et les erreurs stratégiques (inferior) jouent un rôle majeur dans le résultat. Les blunders des noirs diminuent leur probabilité de victoire, tandis que les blunders des blancs augmentent les chances de victoire pour White.
- Les autres variables temporelles et les mouvements longs n'ont qu'un faible impact sur le résultat, à l'exception de quelques cas où les coups longs peuvent influencer légèrement le résultat.

#### Encodage des colonnes non numériques

Un véritable défi dans la préparation des données a été l'encodage des colonnes catégorielles, notamment celles avec un très grand nombre de catégories distinctes. Dans de tels cas, utiliser un encodage classique comme le one-hot encoding aurait été impraticable. Cela aurait non seulement multiplié la taille des données de manière exponentielle, mais également introduit des matrices très clairsemées et difficiles à exploiter efficacement dans les modèles.

De même, l'encodage par label (Label Encoding), qui associe un entier unique à chaque catégorie, aurait pu introduire un biais implicite d'ordre dans certaines configurations de modèles (notamment les modèles linéaires), ce qui n’était pas souhaitable.

Pour surmonter ces limitations, nous avons opté pour le target encoding, une approche plus adaptée. Cette méthode consiste à remplacer chaque catégorie par une valeur numérique calculée en fonction de la cible (par exemple, le taux moyen de victoire pour chaque catégorie). Cela nous a permis de conserver l'information tout en réduisant significativement la complexité des données et en améliorant leur utilisabilité dans les modèles d’apprentissage.

In [ ]:
# Sauver état de df_preparation
df_preparation_original = df_preparation

In [ ]:
# Observation des types des colonnes pour savoir comment les traiter
schema = df_preparation.schema
columns_by_type = defaultdict(list)

for field in schema:
    columns_by_type[str(field.dataType)].append(field.name)

for data_type, columns in columns_by_type.items():
    print(f"Type: {data_type}")
    print(f"Columns: {columns}\n")

Il y a 5 colonnes de type chaine de caractères, nous allons les encoder afin de pouvoir les utiliser dans nos prédictions.

In [ ]:
# Combien de données différentes contiennent chacune de ces colonnes ?
for column in columns_by_type["StringType()"]:
  distinct_count = df_preparation.select(column).distinct().count()
  print(f"Nombre de valeurs distinctes pour la colonne '{column}': {distinct_count}")

Nous allons gérer ces colonnes de façons différentes, en fonction de leur nombre de valeurs possibles et de leur type.

* `Result` : Colonne à prédire (déjà encodée avant l'analyse par corrélation)
* `ECO`, `Opening` : Beaucoup de valeurs possibles
* `Termination`, `Game_type` : Peu de valeurs


In [ ]:
# Supprimer alors Result
df_preparation = df_preparation.drop("Result")

In [ ]:
# Game_type : 4 valeurs, pseudo relation d'ordre
from pyspark.sql.functions import col # important de laisser l'import de nouveau

df_preparation = df_preparation.withColumn(
    "Game_type_encoded",
    when(col("Game_type") == "Bullet", 1)
    .when(col("Game_type") == "Blitz", 2)
    .when(col("Game_type") == "Rapid", 3)
    .when(col("Game_type") == "Classical", 4)
)

# vérifier
df_preparation.select("Game_type", "Game_type_encoded").distinct().show()

In [ ]:
# Supprimer Game_type
df_preparation = df_preparation.drop("Game_type")

In [ ]:
# copy du df pour pas tout perdre
data = df_preparation

In [ ]:
from pyspark.sql.functions import mean
# Target encoding pour "Opening"
avg_opening_result = data.groupBy("Opening").agg(mean("Result_index").alias("opening_score"))
data = data.join(avg_opening_result, on="Opening", how="left")

# Target encoding pour "Eco"
avg_eco_result = data.groupBy("Eco").agg(mean("Result_index").alias("eco_score"))
data = data.join(avg_eco_result, on="Eco", how="left")

# Vérification des colonnes ajoutées
data.select("Opening", "opening_score", "Eco", "eco_score").show(10)

In [ ]:
# Supprimer Opening et Eco
data = data.drop("Opening", "Eco")

In [ ]:
# Quelles sont les valeurs de Termination ?
data.select("Termination").distinct().show()

In [ ]:
# Termination : 4 valeurs possibles
# One hot encoding pour éviter relations d'ordre implicite

# Créer Termination_Abandoned, Termination_Rules_infraction, Termination_Time_forfeit, Termination_Normal

data = data.withColumn(
    "Termination_Abandoned",
    when(col("Termination") == "Abandoned", 1)
    .otherwise(0)
)

data = data.withColumn(
    "Termination_Rules_infraction",
    when(col("Termination") == "Rules infraction", 1)
    .otherwise(0)
)

data = data.withColumn(
    "Termination_Time_forfeit",
    when(col("Termination") == "Time forfeit", 1)
    .otherwise(0)
)

data = data.withColumn(
    "Termination_Normal",
    when(col("Termination") == "Normal", 1)
    .otherwise(0)
)

# Afficher Termination et les 4 colonnes créées
data.select("Termination", "Termination_Abandoned", "Termination_Rules_infraction", "Termination_Time_forfeit", "Termination_Normal").show(10)

In [ ]:
# Supprimer Termination
data = data.drop("Termination")

In [ ]:
# Vérifier les colonnes
data.printSchema()

#### Normalisation des données

In [ ]:
# Quelles sont les colonnes numériques ? (normalement toutes)
numeric_cols = [col[0] for col in data.dtypes if col[1] in ["int", "double"]]
print(f'Les colonnes numériques sont : {numeric_cols}')

# Quelles sont les colonnes non numériques ?
non_numeric_cols = [col[0] for col in data.dtypes if col[1] not in ["int", "double"]]
print(f'Les colonnes non numériques sont : {non_numeric_cols}')

In [ ]:
# Retirer Result_index
numeric_cols.remove("Result_index")

# Combiner les colonnes numériques dans un seul vecteur pour la standardisation
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="numeric_features")
data = assembler.transform(data)

# Standardiser les données
scaler = StandardScaler(inputCol="numeric_features", outputCol="scaled_features", withMean=True, withStd=True)
scaler_model = scaler.fit(data)
data = scaler_model.transform(data)

# Transformer le vecteur en colonnes séparées
data = data.withColumn("scaled_features_array", vector_to_array(col("scaled_features")))

# Réattribuer chaque colonne standardisée à son nom d'origine
for i, col_name in enumerate(numeric_cols):
    data = data.withColumn(f"{col_name}_scaled", col("scaled_features_array")[i])

# Supprimer les colonnes intermédiaires si nécessaire
data = data.drop("numeric_features", "scaled_features", "scaled_features_array")

# Afficher un aperçu des données
data.select(*[f"{col}_scaled" for col in numeric_cols]).show(5)


In [ ]:
# Extraire toutes les colonnes scaled et Result_index
scaled_cols = [col_name for col_name in data.columns if "_scaled" in col_name]
df_scaled = data.select(*scaled_cols, "Result_index")

feature_columns = df_scaled.columns
feature_columns

In [ ]:
feature_columns.remove("Result_index")

In [ ]:
# Combinaison des colonnes finales
assembler_final = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_final = assembler_final.transform(df_scaled)

In [ ]:
df_final.show(5)

In [ ]:
df_features = df_final.select("features", "Result_index")

#### Train test split

In [ ]:
# Diviser les données en 80% pour l'entrainement et 20% pour le test
train_data, test_data = df_features.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Utiliser un sous-ensemble des données (1% des données d'entraînement)
small_train_data = train_data.sample(withReplacement=False, fraction=0.01, seed=1234)


#### Régression multinomiale


Pour entraîner notre modèle de régression multinomiale, nous avons utilisé un sous-échantillon représentant 1 % des données d'entraînement afin de réduire le temps de calcul et d’optimiser les performances. Le modèle a été évalué sur un ensemble de test distinct.

In [ ]:
# Environ 5 minutes avec un sous sample à 1%
lr = LogisticRegression(featuresCol="features", labelCol="Result_index", family="multinomial")
model = lr.fit(small_train_data)

In [ ]:
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="Result_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.4f}")

Le modèle a obtenu une précision (accuracy) de 92,11 %, ce qui signifie que 92 % des prédictions effectuées étaient correctes. Cela indique que le modèle est capable de bien prédire les résultats des parties (victoire des Blancs, victoire des Noirs ou match nul) à partir des caractéristiques utilisées.

Etant donné que les classes dans les données cibles (Result_index) sont relativement équilibrées, la précision est une métrique appropriée pour évaluer la performance du modèle.

$$ \text{Accuracy} = \frac{\text{Nombre de prédictions correctes}}{\text{Nombre total de prédictions}} $$

Ce modèle de régression multinomiale s'est révélé efficace pour prédire le résultat des parties avec des performances élevées, même sur un sous-échantillon réduit des données. Cela valide l’approche choisie et la pertinence des caractéristiques sélectionnées pour la prédiction.


In [ ]:
# Récupérer les noms des caractéristiques utilisées dans le modèle
feature_names = assembler.getInputCols()

In [ ]:
# Intercept pour chaque classe
intercepts = model.interceptVector
print("Intercepts pour chaque classe :")
for i, intercept in enumerate(intercepts):
    print(f"Classe {i}: {intercept}")

In [ ]:
# Coefficient matrix : chaque ligne représente les coefficients pour une classe
coeff_matrix = model.coefficientMatrix
coeff_matrix


In [ ]:
# Afficher les coefficients pour chaque classe
for i in range(coeff_matrix.numRows):
    print(f"Classe {i}:")
    for j in range(coeff_matrix.numCols):
        print(f"  {feature_names[j]}: {coeff_matrix[i, j]}")
    print(f"Intercept: {intercepts[i]}")
    print("\n")

In [ ]:
# Boucle pour afficher les 5 facteurs les plus influents pour chaque classe
for i in range(coeff_matrix.numRows):
    print(f"Classe {i}:")

    # Extraire les coefficients pour la classe i
    class_coeffs = [coeff_matrix[i, j] for j in range(coeff_matrix.numCols)]  # Liste des coefficients

    # Créer une liste de tuples (coefficient, feature_name) et trier par valeur absolue des coefficients
    feature_coeffs = [(feature_names[j], class_coeffs[j]) for j in range(len(class_coeffs))]

    # Trier les caractéristiques par valeur absolue décroissante des coefficients
    feature_coeffs_sorted = sorted(feature_coeffs, key=lambda x: abs(x[1]), reverse=True)

    # Afficher les 5 facteurs les plus influents
    for feature, coeff in feature_coeffs_sorted[:5]:
        print(f"  {feature}: {coeff}")

    # Afficher l'intercept de la classe
    print(f"Intercept: {intercepts[i]}")
    print("\n")


**Classe 0 : White gagne :**

- `black_moves` (-40.48) : Plus le joueur noir joue de coups, moins il est probable que White gagne.
- `white_moves` (40.22) : Plus le joueur blanc joue de coups, plus il est probable qu'il gagne.

  **NB importante :** Les variables `white_moves` et `black_moves` sont naturellement corrélées, car les joueurs alternent leurs coups tout au long de la partie. Un grand nombre de coups totaux implique donc nécessairement une contribution importante des deux joueurs. Cela reflète davantage la durée globale de la partie (longue ou courte) qu'une performance individuelle.

- `Black_blunders` (1.27) : Les erreurs majeures des noirs augmentent significativement la probabilité de victoire pour White.
- `White_blunders` (-1.20) : Les blunders des blancs réduisent la probabilité de victoire pour White.
- `Black_inferior_moves` (0.71) : Les mauvais coups globaux des noirs favorisent également la victoire de White.
- `Intercept` (0.811) : Cela indique une tendance de base légèrement positive pour White, en l'absence de contribution des autres variables.

**Classe 1 : Match nul :**

- `white_moves` (0.64) : Une légère augmentation des mouvements du joueur blanc favorise les matchs nuls. Cet indicateur est peu pertinent, comme déjà expliqué, le nombre de coups joué par white et black est similaire.
- `Termination_Normal` (0.32) : Les parties qui se terminent normalement (pas par abandon ou dépassement de temps) favorisent un match nul. Ce qui est logique.
- `Termination_Time_forfeit` (-0.32) : Un abandon dû au dépassement de temps réduit la probabilité d'un match nul. Ce qui est aussi logique, un forfait implique un gagnant.
- `Game_flips` (0.25) : Les retournements de situation augmentent la probabilité de match nul, ce qui reflète une dynamique équilibrée dans ces parties.
- `Total_moves` (0.22) : Un nombre total élevé de mouvements est associé aux matchs nuls.
- `Intercept` (-1.49) : Cela indique une tendance de base négative pour les matchs nuls, sauf si les variables les favorisent.

**Classe 2 : Black gagne :**

- `white_moves` (-40.86) : Plus le joueur blanc joue de coups, moins il est probable que Black gagne.
- `black_moves` (40.66) : Plus le joueur noir joue de coups, plus il est probable qu'il gagne.

  Comme dit précédemment, les variables `white_moves` et `black_moves` sont naturellement liées.

- `White_blunders` (1.21) : Les erreurs majeures des blancs augmentent significativement la probabilité de victoire pour Black.
- `Black_blunders` (-1.14) : Les blunders des noirs réduisent la probabilité de victoire pour eux.
- `White_inferior_moves` (0.69) : Les mauvais coups globaux des blancs favorisent également la victoire de Black.
- `Intercept` (0.678) : Cela indique une légère tendance de base favorable à Black.

**Conclusion**

Ces résultats mettent en évidence des dynamiques intéressantes dans les parties d'échecs :
- Les blunders et les mauvais coups (inferior moves) sont des indicateurs clés pour prédire le résultat, avec des impacts inverses selon le joueur (Blanc ou Noir).
- Les parties longues et équilibrées semblent favoriser les matchs nuls, tandis que l'activité d'un joueur, mesurée par le nombre de coups, est un facteur déterminant pour les victoires.



#### Conclusion question 3

Les différentes approches ont permis d'identifier les variables significativement liées au Result :

- Corrélation et covariance ont révélé les tendances générales, montrant notamment quels facteurs influencent les chances de victoire pour les Blancs ou les Noirs.
- L'approche par Machine Learning a permis d'aller plus loin, en étudiant également les facteurs associés à une issue de match nul, ce qui n'était pas accessible via des analyses statistiques plus simples.

Ainsi, ces analyses complémentaires offrent une vision plus globale et détaillée des variables influençant l'issue d'une partie.

#### Réponse à l'hypothèse :
- ELO_diff comme facteur prédominant :
Les résultats confirment que la différence d'ELO est effectivement un facteur significatif pour prédire l'issue de la partie, en accord avec la théorie. Une grande différence d'ELO favorise fortement le joueur mieux classé, comme prévu par le modèle de probabilité basé sur une distribution normale.

- Cependant, les résultats montrent que les erreurs (blunders, mistakes) et le contexte (type de partie, dynamique des coups) ajoutent des nuances importantes à cette prédiction. Le facteur temporel lui n'est pas important. Cela suggère que, bien que l'ELO soit un bon indicateur global, l'issue d'une partie peut dépendre de nombreux autres facteurs comportementaux.

## Questions supplémentaires

### Distribution des parties nulles selon l'ouverture et le niveau

- **Quelles ouvertures ont une probabilité plus élevée de mener à une partie nulle ?**
- **Quelle est la distribution des parties nulles en fonction de l'ouverture et des catégories ELO des joueurs ?**

Hypothèses :
- Certaines ouvertures pourraient être plus susceptibles de mener à des parties nulles, par exemple des ouvertures symétriques ou très solides qui tendent à se stabiliser.
- Les joueurs dans les catégories ELO plus élevées pourraient être plus enclins à jouer des ouvertures qui mènent à des positions équilibrées, avec une probabilité plus élevée de nulles.
- Le niveau des joueurs pourrait avoir une influence sur les ouvertures choisies, et des catégories plus faibles (joueurs avec un ELO plus bas) pourraient avoir un taux plus faible de parties nulles, car leurs erreurs stratégiques ou tactiques peuvent rendre la partie plus dynamique et moins susceptible de se terminer par une nulle.
- Le type d'ouverture pourrait aussi influencer le taux de parties nulles, certaines ouvertures plus complexes ou agressives entraînant plus de gains et de pertes, alors que des ouvertures plus passives pourraient mener à des parties plus équilibrées et donc à davantage de nulles.

In [134]:
df_spark_null = df_spark_plus
# ne garder que les données où le Result est 1/2-1/2
df_spark_null = df_spark_null.filter(col("Result") == "1/2-1/2")

In [135]:
# Distribution des parties nulles selon l'ouverture
distribution_null = df_spark_null.groupBy("Opening") \
                                        .agg(count("*").alias("num_draws"),
                                             (count("*") / df_spark.count()).alias("draw_rate")) \
                                        .orderBy(col("draw_rate").desc())


In [136]:
distribution_null.show(truncate=False)

+---------------------------------------------+---------+---------------------+
|Opening                                      |num_draws|draw_rate            |
+---------------------------------------------+---------+---------------------+
|Queen's Pawn Game: Mason Attack              |2459     |6.575026290746647E-4 |
|Indian Game                                  |1859     |4.970709180357062E-4 |
|Sicilian Defense                             |1631     |4.36106867840902E-4  |
|Caro-Kann Defense                            |1552     |4.149833592207725E-4 |
|Philidor Defense                             |1532     |4.096356355194739E-4 |
|Scandinavian Defense: Mieses-Kotroc Variation|1440     |3.850361064935002E-4 |
|French Defense: Knight Variation             |1238     |3.3102409711038426E-4|
|Pirc Defense                                 |1172     |3.1337660889609883E-4|
|Sicilian Defense: Old Sicilian               |1148     |3.0695934045454047E-4|
|Scotch Game                            

Les résultats montrent que certaines ouvertures spécifiques ont une probabilité plus élevée de mener à des parties nulles.

- *Queen's Pawn Game: Mason Attack*, une approche calme et solide, a le taux de nulles le plus élevé, avec un draw rate de 0.065 %.
- Les ouvertures classiques et solides comme *Indian Game* (0.050 %), *Sicilian Defense* (0.044 %), et *Caro-Kann Defense* (0.041 %) figurent également parmi les ouvertures avec les taux de nulles les plus élevés, ce qui confirme leur nature équilibrée et stable.

Ces résultats sont cohérents avec l'hypothèse selon laquelle des ouvertures symétriques ou solides tendent à stabiliser les positions, augmentant ainsi la probabilité d'un match nul.

En revanche, des ouvertures plus agressives ou tactiques, comme celles non mentionnées ici, pourraient entraîner davantage de victoires ou de défaites, réduisant leur draw rate.

In [137]:
# Distribution des parties nulles en fonction de la catégorie ELO des joueurs
distribution_null_by_elo = distribution_null_by_elo = df_spark_null.groupBy("Black_ELO_category", "White_ELO_category") \
                                    .agg(count("*").alias("num_draws"),
                                         (count("*") / df_spark.count()).alias("draw_rate")) \
                                    .orderBy(col("draw_rate").desc())


In [138]:
distribution_null_by_elo.show(truncate = False)

+------------------+------------------+---------+---------------------+
|Black_ELO_category|White_ELO_category|num_draws|draw_rate            |
+------------------+------------------+---------+---------------------+
|Low rating        |Low rating        |67121    |0.017947228127743214 |
|High rating       |High rating       |28864    |0.007717834845714161 |
|High rating       |Low rating        |4569     |0.0012216874795616685|
|Low rating        |High rating       |4257     |0.00113826298982141  |
|GM rating         |GM rating         |2229     |5.960038065097305E-4 |
|GM rating         |High rating       |1254     |3.3530227607142315E-4|
|High rating       |GM rating         |1157     |3.0936581612012485E-4|
|GM rating         |Low rating        |54       |1.4438853993506258E-5|
|Low rating        |GM rating         |43       |1.1497605957792021E-5|
+------------------+------------------+---------+---------------------+



Les résultats montrent que les parties nulles sont plus fréquentes entre joueurs de même niveau, notamment pour les catégories *Low rating* (1,79 %) et *High rating* (0,77 %). Cela peut s'expliquer par un équilibre d'erreurs et de stratégies similaires. En revanche, les *Grands Maîtres* (GM) ont un taux de nulles plus faible (0,59 %) même entre eux, surement car ils savent éviter les positions trop équilibrées.

Au contraire, les matchs entre joueurs de niveaux très différents, comme *GM* vs *Low rating*, présentent des taux de nulles très faibles (inférieur à 0,15 %).

Ces observations soulignent l'influence du niveau des joueurs sur la probabilité d'une partie nulle.

In [139]:
# Croiser les données des ouvertures et des catégories ELO des joueurs pour analyser les probabilités de nulles
distribution_null_by_opening_elo = df_spark_null.groupBy("Opening", "Black_ELO_category", "White_ELO_category") \
                                           .agg(count("*").alias("num_draws"),
                                                (count("*") / df_spark.count()).alias("draw_rate")) \
                                           .orderBy(col("draw_rate").desc())


In [140]:
distribution_null_by_opening_elo.show(truncate = False)

+---------------------------------------------+------------------+------------------+---------+---------------------+
|Opening                                      |Black_ELO_category|White_ELO_category|num_draws|draw_rate            |
+---------------------------------------------+------------------+------------------+---------+---------------------+
|Queen's Pawn Game: Mason Attack              |Low rating        |Low rating        |1878     |5.021512555519399E-4 |
|Philidor Defense                             |Low rating        |Low rating        |1347     |3.601691912824617E-4 |
|Sicilian Defense                             |Low rating        |Low rating        |1167     |3.1203967797077415E-4|
|Scandinavian Defense: Mieses-Kotroc Variation|Low rating        |Low rating        |1049     |2.8048810813311234E-4|
|Caro-Kann Defense                            |Low rating        |Low rating        |1032     |2.759425429870085E-4 |
|Scotch Game                                  |Low ratin

Les résultats montrent que les parties nulles sont particulièrement fréquentes pour des ouvertures classiques et solides, comme la *Queen's Pawn Game: Mason Attack* et la *Philidor Defense*. Ces ouvertures favorisent des positions équilibrées et défensives, ce qui augmente les chances de parties sans vainqueur. On peut tout de même retrouver en troisième position la *Sicilian Defense*, une ouverture asymétrique et aggressive.  
Les joueurs de faible niveau (*Low rating*) dominent ce tableau, confirmant que leurs parties, bien que souvent marquées par des erreurs mutuelles, aboutissent fréquemment à des matchs nuls.

En revanche, les parties impliquant des joueurs de niveaux plus élevés (*High rating*) apparaissent moins souvent dans ce tableau, ici, il n'y en a qu'un sur les vingt premiers. On voit également qu'il n'y a aucun GM dans ces vingt premiers résultats. De plus, nous retrouvons dans ce tableau, seulement des parties avec des joueurs de même niveau.

Ces résultats confirment que les ouvertures et le niveau des joueurs influencent significativement les probabilités de matchs nuls

In [141]:
# TODO : Peut etre changer les hypothèses et y ajouter que un même ELO provoque plus de matchs nuls
# Car les types d'openings ca montre un peu mais pas bcp, donc peut etre compléter jsp

---

### Impact de la différence d'ELO sur la durée d'une partie (en nombre de coups)

Hypothèse à tester : Les parties où la différence d'ELO entre les joueurs est grande ont tendance à durer moins longtemps en nombre de coups. Cela pourrait être dû au fait qu'un joueur plus fort (avec un ELO plus élevé) termine rapidement la partie avec un coup décisif ou un mat, tandis qu'une partie entre des joueurs avec des ELO similaires pourrait durer plus longtemps en raison d'une lutte plus équilibrée.

Pour tester notre hypothèse, nous allons calculer la corrélation entre la différence d'ELO et la durée de la partie.



In [142]:
df_spark_diff_ELO = df_spark

# Ajouter la colonne de différence d'ELO
df_spark_diff_ELO = df_spark_diff_ELO.withColumn("ELO_diff", col("WhiteELO") - col("BlackELO"))

In [143]:
# Créer un VectorAssembler pour combiner les colonnes en un vecteur
assembler = VectorAssembler(inputCols=['ELO_diff', 'Total_moves'], outputCol='features')

# Appliquer le VectorAssembler pour transformer les données
assembled_data = assembler.transform(df_preparation)

# Calcul de la corrélation
correlation_matrix = Correlation.corr(assembled_data, 'features', method='pearson')

# Affichage de la matrice de corrélation
print(f"Correlation Matrix:\n{correlation_matrix.head()}")


Correlation Matrix:
Row(pearson(features)=DenseMatrix(2, 2, [1.0, -0.0077, -0.0077, 1.0], False))


Corrélation très faible : La valeur de la corrélation -0.0077 indique une très faible relation entre la différence d'ELO et le nombre de coups dans la partie. La différence d'ELO entre les joueurs ne semble pas avoir un impact significatif sur la durée de la partie (en termes de nombre de coups).

La corrélation, ayant une valeur proche de 0, signifie qu'il n'y a pas de lien linéaire évident entre ces deux variables. Et la corrélation, étant négative, indique que si un lien existe, il est inverse, c'est-à-dire que des différences d'ELO plus grandes pourraient être associées à des parties plus courtes, mais dans ce cas, l'effet est tellement faible que cela ne constitue pas une relation significative.


**Réflexion sur l'hypothèse :**  
L'hypothèse selon laquelle les parties avec une grande différence d'ELO durent moins longtemps n'est pas confirmée par ces résultats. En effet, la corrélation très faible suggère que d'autres facteurs peuvent jouer un rôle plus important dans la durée de la partie que la différence d'ELO entre les joueurs.

# Conclusion

Ce projet a permis d’explorer en profondeur un grand ensemble de données issues des parties d'échecs jouées sur Lichess.

Grâce aux résultats apportés aux questions proposées et aux problématiques supplémentaires, nous avons pu mettre en lumière plusieurs aspects du jeu d’échecs et offrir des perspectives intéressantes pour mieux comprendre les dynamiques du jeu.

Nous avons constaté que le **taux d’erreurs** (blunders, erreurs, imprécisions) varie significativement selon la catégorie ELO des joueurs, avec des taux plus faibles pour les joueurs de haut niveau.

Nous avons également vu que certaines **ouvertures** offrent des **probabilités de victoire** plus élevées pour les Blancs ou les Noirs en fonction des niveaux de joueurs et des types de parties.

Les modèles prédictifs, en particulier la régression multinomiale, nous ont permis d'atteindre une précision (accuracy) de 92 % dans la **prédiction des résultats** des parties. Bien que ce soit un excellent résultat, il est important de noter que certaines variables, comme la différence d’ELO ou les erreurs commises, montrent une influence limitée lorsqu'elles sont prises isolément. Les interactions complexes entre variables jouent un rôle crucial, et les parties d’échecs restent influencées par des dynamiques stratégiques difficiles à modéliser intégralement.

Ensuite, nous avons analysé la **probabilité de parties nulles** en fonction des ouvertures et des catégories ELO. Les joueurs de même niveau, surtout ceux de faible niveau tendent à produire plus de matchs nuls et utiliser des ouvertures classiques et solides. Et enfin, la **corrélation entre la différence d’ELO et la durée des parties** s’est révélée insignifiante, invalidant l’hypothèse d’une relation directe.

In [144]:
# TODO Zoé : Ajouter ouverture ? j'ai aps d'inspi